## Overview

The EMISOpen consultation writeback is essentially a combination of several HL7 v2 Events.

### ADT_A04 - Register a Patient (patient encounter)

| HL7 v2.4 Segment                                                     | HL7 FHIR Resource                                                                                                    | EMISOpen                                        |
|----------------------------------------------------------------------|----------------------------------------------------------------------------------------------------------------------|-------------------------------------------------|
| [PID](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/PID)   | [Patient.identifier](https://virtually-healthcare.github.io/R4/StructureDefinition-Patient.html)                     | RegistrationType (extends IdentType)            |
| [PV1](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/PV1})  | [Encounter](https://virtually-healthcare.github.io/R4/StructureDefinition-Encounter.html)                            | ConsultationType                                |
| - [OBX](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/OBX) | [Observation](https://virtually-healthcare.github.io/R4/StructureDefinition-Observation.html)                        | EventType (both coded and uncoded)              |
| OBX (type = ED)                                                      | [DocumentReference](https://virtually-healthcare.github.io/R4/StructureDefinition-DocumentReference.html) and Binary | Attachment                                      |
| OBX (dependent on code)                                              | [Task](https://virtually-healthcare.github.io/R4/StructureDefinition-Task.html)                                      | DiaryType                                       |
| OBX (dependent on code)                                              | [Immunization](https://virtually-healthcare.github.io/R4/StructureDefinition-Immunization.html)                      | EventType                                       |
| - [AL1](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/AL1) | [AllergyIntolerance](https://virtually-healthcare.github.io/R4/StructureDefinition-AllergyIntolerance.html)                                                                                              | AllergyType                                     |
| - [DG1](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/DG1) | [Condition](https://virtually-healthcare.github.io/R4/StructureDefinition-Condition.html)                            | EventType (with problem elements)               |
| - [PR1](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/PR1) | [Procedure](https://virtually-healthcare.github.io/R4/StructureDefinition-Procedure.html)                            | EventType (can be inferred from SNOMED CT code) |

### RDE_O11 - Pharmacy/treatment encoded order

| HL7 v2.4 Segment                                                    | HL7 FHIR Resource                                                                                         | EMISOpen                             |
|---------------------------------------------------------------------|-----------------------------------------------------------------------------------------------------------|--------------------------------------|
| [PID](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/PID)  | [Patient.identifier](https://virtually-healthcare.github.io/R4/StructureDefinition-Patient.html)          | RegistrationType (extends IdentType) |
| [PV1](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/PV1}) | [Encounter](https://virtually-healthcare.github.io/R4/StructureDefinition-Encounter.html)                 | ConsultationType                     |
| [RXE](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/RXE)  | [MedicationRequest](https://virtually-healthcare.github.io/R4/StructureDefinition-MedicationRequest.html) | MedicationType                       |



### Miscellaneous HL7 v2 Segments

| HL7 v2.4 Segment                                                   | HL7 FHIR Resource                                                                                       | EMISOpen          |
|--------------------------------------------------------------------|---------------------------------------------------------------------------------------------------------|-------------------|
| [ORC](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/ORC) | [ServiceRequest](https://virtually-healthcare.github.io/R4/StructureDefinition-ServiceRequest.html)     | ReferralType      |
| [OBR](https://hl7-definition.caristix.com/v2/HL7v2.4/Segments/ORC) | [DiagnosticReport](https://virtually-healthcare.github.io/R4/StructureDefinition-DiagnosticReport.html) | InvestigationType |

This is not a 100% accurate map, but we can see HL7 v2, FHIR and EMIS are related. Similar maps can be created to other GP systems, and they are similar. One of the main reasons for this is UK GP systems were based on early HL7, so they also share similar models with many secondary care systems and laboratory systems.

We will use FHIR as:

1. We internally use FHIR as our [Canonical Data Model](https://www.enterpriseintegrationpatterns.com/patterns/messaging/CanonicalDataModel.html). The link provides a brief description of the decision behind this (the book has a more comprehensive description).
2. The tooling to support FHIR models is quite extensive.

## HL7 v2 vrs FHIR

We don't view v2 and FHIR as competing. HL7 v2 is primarily a workflow interaction standard that supports asynchronous input/output. FHIR, on the other hand, is more focused on REST API interactions for querying data, which is something HL7 v2 doesn't support (well). As mentioned previously, we also use FHIR as the master data model as it can be converted to other standards such as HL7 v2 and XDS quite easily.





## Consultation Writeback

The writeback is a complex workflow. The diagram below is the intitial design for the updated writeback which includes the ability to send to GP Practices which don't use virtually via GP Connect Send Document.

![Alt text](../images/Writeback.drawio.png)

### Application - Clinical Process

A practitioner:

- GP - creates a clinical management plan
- Pharmacist - creates a consultation note

Both are stored in the Clinical Data Repository (FHIR Repository) and then a FHIR Task is created with status `requested`, requesting the consultation to be sent to the patient's GP.

### Workflow

Tasks are polled from the FHIR Repository. For each Task:

- Read consultation from the FHIR Repository and update FHIR Task to `accepted`
- Lookup Patients GP from PDS (**NEW**) and then get Endpoint details to check to see if this is a local delivery.
- If EMIS
  - Convert to EMISOpen
  - Send consultation to EMIS via IM1 Transaction
- elseIf TPP
  - Convert to TPP
  - Send consultation to TPP via IM1 Transaction
- Else
  - Convert to a FHIR Document (**NEW**)
  - Convert FHIR Document to PDF (**NEW**)
  - Send consultation to GP Practice via MESH (workflow: GP Connect)
- Update FHIR Task to `completed`.

Note: For conformance reasons, a pharmacist should not use IM1 Transaction for

Exceptions result in the FHIR Task status being changed to `failed`


### Example set up

This gets a previously completed task and resets it

1. This is `GET Task\8b12d352-b2dc-4647-a544-ea2ed30011e1`
2. update `status` to `accepted`
3. Update the Task via `POST Task\8b12d352-b2dc-4647-a544-ea2ed30011e1`

In [52]:
import requests
import json

headersCDR = {"Content-Type": "application/fhir+json"}

cdrFHIRUrl = "http://localhost:8180/CDR/FHIR/R4"


parameters = {}

tasks = []
response = requests.get(cdrFHIRUrl + '/Task/8b12d352-b2dc-4647-a544-ea2ed30011e1',parameters,headers=headersCDR)
if response.status_code == 200:
    taskJSON = json.loads(response.text)
    taskJSON['status'] = 'accepted'
    response = requests.put(cdrFHIRUrl + '/Task/8b12d352-b2dc-4647-a544-ea2ed30011e1',json.dumps(taskJSON),headers=headersCDR)
    print(response.text)


{
  "resourceType": "Task",
  "id": "8b12d352-b2dc-4647-a544-ea2ed30011e1",
  "meta": {
    "versionId": "37",
    "lastUpdated": "2025-06-20T09:48:42.489Z"
  },
  "identifier": [ {
    "system": "https://fhir.virtually.healthcare/Id/Task",
    "value": "537fd315-8a60-4c02-a96e-473bd04a138d"
  } ],
  "status": "accepted",
  "intent": "order",
  "priority": "routine",
  "code": {
    "coding": [ {
      "system": "http://fhir.virtuallyhealthcare.co.uk/CodeSystem/tasks",
      "code": "WRITEBACK",
      "display": "Consultation Write Back"
    } ]
  },
  "focus": {
    "reference": "Encounter/3493d264-ab4e-4794-9622-555ed16b75aa",
    "identifier": {
      "system": "https://fhir.virtually.healthcare/Id/Encounter",
      "value": "d3aa0f25-6ceb-4baf-bb4f-8f53170c5231"
    }
  },
  "for": {
    "reference": "Patient/2cdce27e-9398-4367-8719-d872710e8e63"
  },
  "authoredOn": "2025-05-19T15:43:08+00:00",
  "owner": {
    "reference": "Organization/61a55cac-08d5-4cf8-a4d5-581f50962a82",
    

## Get writeback tasks

In [53]:
import requests
import json

headersCDR = {"Content-Type": "application/fhir+json"}

cdrFHIRUrl = "http://localhost:8180/CDR/FHIR/R4"


parameters = {'_sort' : '-authored-on',
              'authored-on': 'gt2023-07-24',
              'status': 'accepted'}

tasks = []
response = requests.get(cdrFHIRUrl + '/Task',parameters,headers=headersCDR)
if response.status_code == 200:
    tasksJSON = json.loads(response.text)
    if 'entry' in tasksJSON:
        for entry in tasksJSON['entry']:
            if 'resource' in entry:
                print(entry['resource'])
                tasks.append(entry['resource'])


{'resourceType': 'Task', 'id': '8b12d352-b2dc-4647-a544-ea2ed30011e1', 'meta': {'versionId': '37', 'lastUpdated': '2025-06-20T09:48:42.489Z'}, 'identifier': [{'system': 'https://fhir.virtually.healthcare/Id/Task', 'value': '537fd315-8a60-4c02-a96e-473bd04a138d'}], 'status': 'accepted', 'intent': 'order', 'priority': 'routine', 'code': {'coding': [{'system': 'http://fhir.virtuallyhealthcare.co.uk/CodeSystem/tasks', 'code': 'WRITEBACK', 'display': 'Consultation Write Back'}]}, 'focus': {'reference': 'Encounter/3493d264-ab4e-4794-9622-555ed16b75aa', 'identifier': {'system': 'https://fhir.virtually.healthcare/Id/Encounter', 'value': 'd3aa0f25-6ceb-4baf-bb4f-8f53170c5231'}}, 'for': {'reference': 'Patient/2cdce27e-9398-4367-8719-d872710e8e63'}, 'authoredOn': '2025-05-19T15:43:08+00:00', 'owner': {'reference': 'Organization/61a55cac-08d5-4cf8-a4d5-581f50962a82', 'identifier': {'system': 'https://fhir.nhs.uk/Id/ods-organization-code', 'value': 'F83004'}}, 'note': [{'time': '2025-05-19T15:44:09

### Process Tasks

1. Extract the full encounter from the CDR
2. TODO Use FHIR Validation to ensure it is valid. If not reject (and set Task.status to
4. Convert to EMISOpen
5. TODO Send to EMIS
6. Update Task

In [54]:

emisFHIRUrl = "http://localhost:8180/EMIS/FHIR/R4"
headersEMIS = {"Content-Type": "application/fhir+json",
               "ODS_CODE": "F83004"}

for task in tasks:
    encounter = task['focus']['identifier']
    print(encounter)
    parameters = {'identifier' : encounter['system'] + '|' + encounter['value']}


    responseCDR = requests.get(cdrFHIRUrl + '/Encounter/$extract-collection',parameters,headers=headersCDR)
    if responseCDR.status_code == 200:
        print("======= Response from extract collection ========")
        print(responseCDR.text)
        responseEMIS = requests.post(emisFHIRUrl + '/Bundle/$transform-EMISOpen', responseCDR.text, headers=headersEMIS )
        print("======= Response from transform to EMIS Open ========")
        print(responseEMIS.text)



{'system': 'https://fhir.virtually.healthcare/Id/Encounter', 'value': 'd3aa0f25-6ceb-4baf-bb4f-8f53170c5231'}
======= Response from extract collection ========
{
  "resourceType": "Bundle",
  "type": "collection",
  "entry": [ {
    "fullUrl": "Patient/2cdce27e-9398-4367-8719-d872710e8e63/_history/1",
    "resource": {
      "resourceType": "Patient",
      "id": "2cdce27e-9398-4367-8719-d872710e8e63",
      "meta": {
        "versionId": "1",
        "lastUpdated": "2025-05-19T15:31:18.160Z"
      },
      "identifier": [ {
        "use": "usual",
        "type": {
          "coding": [ {
            "system": "http://terminology.hl7.org/CodeSystem/v2-0203",
            "code": "MR"
          } ]
        },
        "system": "https://emis.com/Id/Patient/DBID",
        "value": "500837",
        "assigner": {
          "identifier": {
            "system": "https://fhir.nhs.uk/Id/ods-organization-code",
            "value": "F83004"
          }
        }
      }, {
        "use": "offi